# Disparate Impact Remover ~ Race

In [142]:
'''
- Importing Libraries
- Filtering Warnings
- Setting random seed
'''

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')


np.random.seed(1)

In [143]:
# Load data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')

In [144]:
'''
- Define target
- Set privileged and unprivileged groups
'''
target = 'two_year_recid'

privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

In [145]:
# Creating BinaryLabelDataset to calculate disparate impact in training data
binary_label_dataset_original = BinaryLabelDataset(
    df=train, 
    label_names=[target], 
    protected_attribute_names=['race'],
    favorable_label=0, 
    unfavorable_label=1,
)

di = BinaryLabelDatasetMetric(
    binary_label_dataset_original,
    unprivileged_groups=unprivileged_groups,
    privileged_groups=privileged_groups
).disparate_impact()

print("Disparate Impact:", di)

Disparate Impact: 0.7885148339939714


In [146]:
# Apply Disparate Impact Remover (DIR)
def apply_dir(rep_lev, bin_label_data):
    di_remover = DisparateImpactRemover(repair_level=rep_lev)
    binary_label_dataset_transformed = di_remover.fit_transform(bin_label_data)
    df_transformed = binary_label_dataset_transformed.convert_to_dataframe()[0] 
    return df_transformed

df_transformed = apply_dir(0.5, binary_label_dataset_original)

In [147]:
# Logistic regression model function
def logistic_model(x_train_transformed, y_train_transformed, x_test_transformed):
    lr_params = {
        "penalty": "l2", 
        "solver": "liblinear",  
        "random_state": rand_seed,
    }
    lr_dir = Pipeline(
        steps=[
            ("preprocessing", StandardScaler()),
            ("classifier", LogisticRegression(**lr_params)), 
        ]
    )
    lr_dir.fit(x_train_transformed, y_train_transformed.values.ravel())
    y_pred_test_lr_dir = lr_dir.predict(x_test_transformed)
    print(f'Test Accuracy: {accuracy_score(y_test, y_pred_test_lr_dir)}')
    print(confusion_matrix(y_test, y_pred_test_lr_dir))
    return y_pred_test_lr_dir

In [148]:
# Generate disparate impact results
def gen_dir_results(test_df, all_results):
    binary_label_dataset_original_test = BinaryLabelDataset(
        df=test_df, 
        label_names=[target], 
        protected_attribute_names=['race'],
        favorable_label=0, 
        unfavorable_label=1
    )
    dir_metrics_original_test = BinaryLabelDatasetMetric(
        binary_label_dataset_original_test,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    ).disparate_impact()
    print("Disparate Impact in original test set:", dir_metrics_original_test)

    binary_label_dataset_lr_dir = BinaryLabelDataset(
        df=all_results, 
        label_names=['y_pred_test_lr_dir'], 
        protected_attribute_names=['race'],
        favorable_label=0, 
        unfavorable_label=1
    )
    dir_metrics_lr_dir = BinaryLabelDatasetMetric(
        binary_label_dataset_lr_dir,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    ).disparate_impact()
    print("Disparate Impact in predictions:", dir_metrics_lr_dir)

In [150]:
X_train = df_transformed.drop(columns=[target])
y_train = df_transformed[target]
X_test = test.drop(columns=[target])
y_test = test[target]

# Fit model, predict and compile results
y_pred_test_lr_dir = logistic_model(X_train, y_train, X_test)
all_results = pd.concat([test[['race', target]].reset_index(drop=True),
                         pd.Series(y_pred_test_lr_dir, name="y_pred_test_lr_dir")], axis=1)
gen_dir_results(test, all_results)

Test Accuracy: 0.6502525252525253
[[288 127]
 [150 227]]
Disparate Impact in original test set: 0.6856802208835342
Disparate Impact in predictions: 0.5744057310322371
